<a href="https://colab.research.google.com/github/sePerezAlbor/LAB1_EDD2_202330/blob/main/LAB_1_DATOS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LABORATORIO 1.**
**Integrantes:**
*  Kanery Marcela Camargo Rodríguez
*  Emanuel de Jesús Carbonell Naranjo.
*  Sebastián Alfonso Pérez Albor.

###  **LIBRERIAS.**

In [878]:
#!apt-get install -y graphviz-dev
#!pip install pygraphviz
import pygraphviz as pgv
from PIL import Image
from IPython.display import Image, display
from typing import Any, List, Optional,Tuple
import pandas as pd
import folium
import copy
from IPython.display import HTML

###  **CLASE COLA.**

In [879]:
class Queue:
    def __init__(self, size: int) -> None:
        self.queue: List[Any] = []
        self.size = size


    def __repr__(self) -> str:
        return str(self.queue)

    def __len__(self) -> int:
        return len(self.queue)

    def add(self, elem: int) -> None:
        if len(self.queue) >= self.size:
            raise ValueError('The Queue is full')

        self.queue.append(elem)

    def is_empty(self) -> bool:
        return len(self.queue) == 0

    def reversed_queue(self):
        if not self.queue:
            raise ValueError('The Queue is empty')

        queue_aux = Queue(len_rr := len(self.queue))
        for _ in range(len_rr):
            val_aux = self.queue.pop()
            queue_aux.add(val_aux)
        self.queue = queue_aux
        del(queue_aux,len_rr)

    def remove(self) -> int:
        if not self.queue:
            raise ValueError('The Queue is empty')

        return self.queue.pop(0)

### **NODO AVL**

In [880]:
metricas_asignadas = []                                     # Lista para guardar metricas asignadas
metricas_repetidas = []                                     # Lista para guardar metricas repetidas
class NodoAVL:
    def __init__(self,
                 titulo:str,
                 departamento: str,
                 ciudad: str,
                 tipo_inmueble: str,
                 latitud: float,
                 longitud:float,
                 superficie_total: float,
                 superficie_cubierta: float,
                 cuartos: float,
                 banos:float,
                 tipo_operacion: str,
                 precio: float
                 ) -> None:


        metrica = round(precio/superficie_total)
        self.metrica_repetida = False
        self.metrica_original = metrica
        ciclo = 0
        while (metrica in metricas_asignadas):
            if ciclo == 0:
                metricas_repetidas.append(metrica)
                self.metrica_repetida = True
            metrica += cuartos                              # Segunda metrica es la primera mas el número de cuartos, solo en caso de que la primera ya haya sido asignada
            ciclo   += 1


        metricas_asignadas.append(metrica)

        self.metrica = metrica                              # Primera metrica
        self.izquierda: Optional['NodoAVL'] = None          # Hijo izquierdo
        self.derecha: Optional['NodoAVL'] = None            # Hijo derecho
        self.altura = 1                                     # Inicialmente, la altura del nodo es 1 SOLO PARA EL BALANCEO

        self.info = {
            'titulo': titulo,                               # Titulo del inmueble
            'departamento': departamento,                   # Departamento del inmueble
            'ciudad': ciudad,                               # Ciudad del inmueble
            'tipo_inmueble': tipo_inmueble,                 # Tipo de inmueble
            'latitud': latitud,                             # Latitud del inmueble
            'longitud': longitud,                           # Longitud del inmueble
            'superficie_total': superficie_total,           # Superficie total del inmueble
            'superficie_cubierta': superficie_cubierta,     # Superficie cubierta del inmueble
            'cuartos': cuartos,                             # Cuartos del inmueble
            'banos': banos,                                 # Baños del inmueble
            'tipo_operacion': tipo_operacion,               # Tipo de operacion del inmueble
            'precio': precio                                # Precio del inmbuele
        }


### **ÁRBOL AVL Y LOS RESPECTIVOS MÉTODOS USADOS.**

In [881]:
lista_nodos = []
class ArbolAVL:
    def __init__(self):
        self.raiz = None

    def altura(self, nodo):
        if nodo is None:
            return 0
        return nodo.altura


    def actualizar_altura(self, nodo):
        nodo.altura = 1 + max(self.altura(nodo.izquierda), self.altura(nodo.derecha))

    def rotacion_derecha(self, y):
        x = y.izquierda
        t2 = x.derecha

        x.derecha = y
        y.izquierda = t2
        self.actualizar_altura(y)
        self.actualizar_altura(x)

        return x

    def rotacion_izquierda(self, x):
        y = x.derecha
        t2 = y.izquierda

        y.izquierda = x
        x.derecha = t2
        self.actualizar_altura(x)
        self.actualizar_altura(y)

        return y

    def balance(self, nodo):
        if nodo is None:
            return 0
        return self.altura(nodo.izquierda) - self.altura(nodo.derecha)

    def __insertar(self, nodo, nuevo_nodo):
        if nodo is None:
            return nuevo_nodo

        if nuevo_nodo.metrica < nodo.metrica:
            nodo.izquierda = self.__insertar(nodo.izquierda, nuevo_nodo)
        else:
            nodo.derecha = self.__insertar(nodo.derecha, nuevo_nodo)

        self.actualizar_altura(nodo)

        factor_balance = self.balance(nodo)

        # Caso izquierda
        if factor_balance > 1 and nuevo_nodo.metrica < nodo.izquierda.metrica:
            return self.rotacion_derecha(nodo)

        # Caso derecha
        if factor_balance < -1 and nuevo_nodo.metrica > nodo.derecha.metrica:
            return self.rotacion_izquierda(nodo)

        # Caso izquierda-derecha
        if factor_balance > 1 and nuevo_nodo.metrica > nodo.izquierda.metrica:
            nodo.izquierda = self.rotacion_izquierda(nodo.izquierda)
            return self.rotacion_derecha(nodo)

        # Caso derecha-izquierda
        if factor_balance < -1 and nuevo_nodo.metrica < nodo.derecha.metrica:
            nodo.derecha = self.rotacion_derecha(nodo.derecha)
            return self.rotacion_izquierda(nodo)

        return nodo

    def insertar(self, nuevo_nodo, modo = 0):
        self.raiz = self.__insertar(self.raiz, nuevo_nodo)
        if modo == 1:
            self.graficar_arbol()

    def preorden(self, nodo):
        if nodo:
            print(nodo.metrica, end=" ")
            self.preorden(nodo.izquierda)
            self.preorden(nodo.derecha)

    def imprimir_preorden(self):
        self.preorden(self.raiz)


    def eliminar(self, nodo, metrica):
        if nodo is None:
            return nodo

        if metrica < nodo.metrica:
            nodo.izquierda = self.eliminar(nodo.izquierda, metrica)
        elif metrica > nodo.metrica:
            nodo.derecha = self.eliminar(nodo.derecha, metrica)
        else:
            # Caso cuando se encuentra el nodo a eliminar
            if nodo.izquierda is None:
                temp = nodo.derecha
                nodo = None
                return temp
            elif nodo.derecha is None:
                temp = nodo.izquierda
                nodo = None
                return temp
            # Caso cuando el nodo tiene dos hijos
            temp = self.minimo_nodo(nodo.derecha)
            nodo.metrica = temp.metrica
            nodo.derecha = self.eliminar(nodo.derecha, temp.metrica)

        self.actualizar_altura(nodo)

        balance = self.balance(nodo)

        # Casos de balanceo similares a la inserción
        if balance > 1 and self.balance(nodo.izquierda) >= 0:
            return self.rotacion_derecha(nodo)

        if balance < -1 and self.balance(nodo.derecha) <= 0:
            return self.rotacion_izquierda(nodo)

        if balance > 1 and self.balance(nodo.izquierda) < 0:
            nodo.izquierda = self.rotacion_izquierda(nodo.izquierda)
            return self.rotacion_derecha(nodo)

        if balance < -1 and self.balance(nodo.derecha) > 0:
            nodo.derecha = self.rotacion_derecha(nodo.derecha)
            return self.rotacion_izquierda(nodo)


        return nodo

    def eliminar_por_metrica(self, metrica):
        self.raiz = self.eliminar(self.raiz, metrica)
        self.graficar_arbol()


    def minimo_nodo(self, nodo):                            # Predecesor
        actual = nodo
        while actual.izquierda is not None:
            actual = actual.izquierda
        return actual


    def graficar_arbol(self):
        graph = pgv.AGraph(directed=True)

        # Función recursiva para agregar nodos y aristas al gráfico
        def agregar_nodos_y_aristas(nodo, padre=None):
            if nodo:
                # Agrega el nodo actual al gráfico
                etiqueta = f"Ciudad: {nodo.info['ciudad']}\nTipo: {nodo.info['tipo_inmueble']}\nPrecio: {int(nodo.info['precio']):,} $\nSuperficie Total: {nodo.info['superficie_total']} m2\nCuartos: {int(nodo.info['cuartos'])}\nMétrica: {int(nodo.metrica_original):,} $/m2"

                graph.add_node(nodo.metrica, label= etiqueta)

                # Si hay un padre, agrega una arista desde el padre al nodo actual
                if padre:
                    graph.add_edge(padre.metrica, nodo.metrica)

                # Llama recursivamente a la función para los hijos izquierdo y derecho
                agregar_nodos_y_aristas(nodo.izquierda, nodo)
                agregar_nodos_y_aristas(nodo.derecha, nodo)

        agregar_nodos_y_aristas(self.raiz)
        graph.draw("arbol.png", format='png', prog='dot')
        #display(Image(data=graph.draw(format='png', prog='dot')))




    # RECORRIDO POR NIVELES.
    def recorrido_niveles_recursivo(self, modo = 0):
        lista_nodos = []
        altura = self.raiz.altura
        if modo == 0:
            print("La altura del árbol es igual a: ", altura)

        for i in range(1, altura + 1):
            self._recorrido_niveles_recursivo(self.raiz, i, modo)

    def _recorrido_niveles_recursivo(self, nodo, level, modo):
        if nodo is None:
            return
        if level == 1:
            if modo == 0:
                print(nodo.metrica_original, end= " ")
            else:
                lista_nodos.append(nodo)

        elif level > 1:
            self._recorrido_niveles_recursivo(nodo.izquierda, level - 1, modo)
            self._recorrido_niveles_recursivo(nodo.derecha, level - 1, modo)

    def buscar_por_metrica(self, metrica):
        return self.__buscar_por_metrica(self.raiz, metrica)


    def __buscar_por_metrica(self, nodo, metrica):
        if nodo is None:
            return None
        if metrica == nodo.metrica:
            return nodo
        elif metrica < nodo.metrica:
            return self.__buscar_por_metrica(nodo.izquierda, metrica)
        else:
            return self.__buscar_por_metrica(nodo.derecha, metrica)

    def calcular_nivel(self, metrica):
        return self.__calcular_nivel(self.raiz, metrica, 1)

    def __calcular_nivel(self, nodo, metrica, nivel_actual):
        if nodo is None:
            return None

        if metrica == nodo.metrica:
            return nivel_actual
        elif metrica < nodo.metrica:
            return self.__calcular_nivel(nodo.izquierda, metrica, nivel_actual + 1)
        else:
            return self.__calcular_nivel(nodo.derecha, metrica, nivel_actual + 1)


    def factor_balanceo(self, nodo):
        if nodo is None:
            return 0

        # Obtener la altura del subárbol izquierdo y derecho.
        alturai = nodo.izquierda.altura if nodo.izquierda else 0
        alturad = nodo.derecha.altura if nodo.derecha else 0

        # Calcular el factor de balanceo.
        balance_factor = alturad - alturai

        return balance_factor

    def obtener_balance_factor(self, metrica):
        return self.__obtener_balance_factor(self.raiz, metrica) * -1

    def __obtener_balance_factor(self, nodo, metrica):
        if nodo is None:
            return None

        if metrica == nodo.metrica:
            return self.balance(nodo)
        elif metrica < nodo.metrica:
            return self.__obtener_balance_factor(nodo.izquierda, metrica)
        else:
            return self.__obtener_balance_factor(nodo.derecha, metrica)


    def buscar_abuelo_padre_tio(self, metrica):
        return self.__buscar_abuelo_padre_tio(self.raiz, None, metrica)

    def __buscar_abuelo_padre_tio(self, nodo, padre, metrica):
        if nodo is None:
            return None, None, None

        if metrica == nodo.metrica:
            if padre is None:
                return None, None, None  # El nodo raíz no tiene padre, abuelo ni tío.
            abuelo = self.__buscar_abuelo(self.raiz, padre.metrica)
            if abuelo:
                if abuelo.izquierda == padre:
                    tio = abuelo.derecha
                else:
                    tio = abuelo.izquierda
            else:
                tio = None
            return abuelo, padre, tio
        elif metrica < nodo.metrica:
            return self.__buscar_abuelo_padre_tio(nodo.izquierda, nodo, metrica)
        else:
            return self.__buscar_abuelo_padre_tio(nodo.derecha, nodo, metrica)

    def __buscar_abuelo(self, nodo, metrica):
        if nodo is None:
            return None
        if (nodo.izquierda and nodo.izquierda.metrica == metrica) or (nodo.derecha and nodo.derecha.metrica == metrica):
            return nodo

        if metrica < nodo.metrica:
            return self.__buscar_abuelo(nodo.izquierda, metrica)
        else:
            return self.__buscar_abuelo(nodo.derecha, metrica)



    def insertar_nodo_desde_usuario(self):
        while True:
            try:
                # Solicitar los datos necesarios al usuario
                titulo = input("Ingrese el título del inmueble: ")
                departamento = input("Ingrese el departamento: ")
                ciudad = input("Ingrese la ciudad: ")
                tipo_inmueble = input("Ingrese el tipo de inmueble: ")
                latitud = float(input("Ingrese la latitud: "))
                longitud = float(input("Ingrese la longitud: "))
                superficie_total = float(input("Ingrese la superficie total: "))
                superficie_cubierta = float(input("Ingrese la superficie cubierta: "))
                cuartos = float(input("Ingrese el número de cuartos: "))
                banos = float(input("Ingrese el número de baños: "))
                tipo_operacion = input("Ingrese el tipo de operación: ")
                precio = float(input("Ingrese el precio: "))

                # Validar que los datos sean del tipo correcto
                if not isinstance(latitud, float) or not isinstance(longitud, float) or \
                   not isinstance(superficie_total, float) or not isinstance(superficie_cubierta, float) or \
                   not isinstance(cuartos, float) or not isinstance(banos, float) or \
                   not isinstance(precio, float):
                    raise ValueError("Algunos de los datos ingresados no son números válidos.")

                # Crear un nuevo nodo con los datos ingresados
                nuevo_nodo = NodoAVL(
                    titulo,
                    departamento,
                    ciudad,
                    tipo_inmueble,
                    latitud,
                    longitud,
                    superficie_total,
                    superficie_cubierta,
                    cuartos,
                    banos,
                    tipo_operacion,
                    precio
                )

                # Insertar el nuevo nodo en el árbol AVL
                self.insertar(nuevo_nodo, modo = 1)
                break  # Salir del bucle si los datos son válidos y el nodo se ha insertado correctamente

            except ValueError as e:
                print(f"Error: {e}. Intente nuevamente.")


### **IMPORTAR LA BASE DE DATOS.**

In [882]:
propietarios = pd.read_csv("propietarios.csv")
#propietarios.columns = ['TITULO','DEPARTAMENTO','CIUDAD','TIPO_INMUEBLE','LATITUD','LONGITUD','SUPERFICIE_TOTAL','SUPERFICIE_CUBIERTA','CUARTOS','BAÑOS','TIPO_OPERACION','PRECIO']
#propietarios

### **CREAR EL ARBOL**

In [883]:
arbol_propietarios = ArbolAVL()
matriz_proprietarios = propietarios.values.tolist()
for fila in matriz_proprietarios:
    nodo = NodoAVL(fila[0],fila[1],fila[2],fila[3],fila[4],fila[5],fila[6],fila[7],fila[8],fila[9],fila[10],fila[11])
    arbol_propietarios.insertar(nodo)

### **GRAFICAR EL ARBOL.**

In [884]:
arbol_propietarios.graficar_arbol()

In [885]:
prelista_nodos = []
def menu():
    print(" ")
    print("Menú de Opciones:")
    print("1. Insertar un nodo.")
    print("2. Buscar un nodo por su métrica.")
    print("3. Eliminar un nodo por su métrica")
    print("4. Buscar un nodo por la serie de criterios")
    print("5. Mostrar árbol por niveles.")
    print("6. Salir")

def menu_opcion4():
    global lista_nodos
    lista_nodos = []
    arbol_propietarios.recorrido_niveles_recursivo(modo=1)
    lista_encontrados_criterios = []
    numero_criterios = int(input("Ingrese el número de criterios que se van a tener en cuenta, recuerde que es de 1 a 3 maximo: "))
    for i in range(numero_criterios):
        print(f"Digite 1 si el criterio {i+1} es: TITULO, DEPARTAMENTO, CIUDAD, TIPO DE INMUEBLE, TIPO DE OPERACIÓN")
        print(f"Digite 2 si el criterio {i+1} es: LATITUD, LONGITUD, SUPERFICIE TOTAL, SUPERFICIE CUBIERTA, CUARTOS, BAÑOS, PRECIO")
        criterio = int(input(""))
        if criterio == 1:
            poc = int(input(" 1: TITULO, 2: DEPARTAMENTO, 3: CIUDAD, 4: TIPO DE INMUEBLE, 5: TIPO DE OPERACIÓN: "))
            cond = input("Ingrese la condición: ")

            if poc == 1:
                for nodo in lista_nodos:
                    if cond in nodo.info['titulo']:
                        lista_encontrados_criterios.append(nodo)

            elif poc == 2:
                for nodo in lista_nodos:
                    if cond.casefold() == nodo.info['departamento'].casefold():
                        lista_encontrados_criterios.append(nodo)

            elif poc == 3:
                for nodo in lista_nodos:
                    if cond.casefold() == nodo.info['ciudad'].casefold():
                        lista_encontrados_criterios.append(nodo)

            elif poc == 4:
                for nodo in lista_nodos:
                    if cond.casefold() == nodo.info['tipo_inmueble'].casefold():
                        lista_encontrados_criterios.append(nodo)

            else:
                for nodo in lista_nodos:
                    if cond.casefold() == nodo.info['tipo_operacion'].casefold():
                        lista_encontrados_criterios.append(nodo)
        else:
            poc = int(input(" 1: LATITUD, 2: LONGITUD, 3: SUPERFICIE TOTAL, 4: SUPERFICIE CUBIERTA, 5: CUARTOS, 6: BAÑOS, 7: PRECIO: "))
            cond = int(input("1: <, 2: >, 3: <=, 4. >=, 5 = : "))
            valorcod = int(input("Ingrese valor: "))

            if poc == 1:
                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['latitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['latitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['latitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['latitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['latitud']:
                        lista_encontrados_criterios.append(nodo)

            elif poc == 2:
                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['longitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['longitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['longitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['longitud']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['longitud']:
                        lista_encontrados_criterios.append(nodo)

            elif poc == 3:
                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['superficie_total']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['superficie_total']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['superficie_total']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['superficie_total']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['superficie_total']:
                        lista_encontrados_criterios.append(nodo)

            elif poc == 4:
                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['superficie_cubierta']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['superficie_cubierta']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['superficie_cubierta']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['superficie_cubierta']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['superficie_cubierta']:
                        lista_encontrados_criterios.append(nodo)


            elif poc == 5:

                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['cuartos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['cuartos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['cuartos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['cuartos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['cuartos']:
                        lista_encontrados_criterios.append(nodo)

            elif poc == 6:
                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['banos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['banos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['banos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['banos']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['banos']:
                        lista_encontrados_criterios.append(nodo)

            else:
                for nodo in lista_nodos:
                    if cond == 1 and valorcod > nodo.info['precio']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 2 and valorcod < nodo.info['precio']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 3 and valorcod >= nodo.info['precio']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 4 and valorcod <= nodo.info['precio']:
                        lista_encontrados_criterios.append(nodo)
                    elif cond == 5 and valorcod == nodo.info['precio']:
                        lista_encontrados_criterios.append(nodo)

        lista_nodos = copy.deepcopy(lista_encontrados_criterios)
        lista_encontrados_criterios = []

    #return lista_encontrados_criterios

while True:

    menu()
    opcion = input("Seleccione una opción (1/2/3/4/5/6): ")

    # Validar si la opción es un número dentro del rango deseado (1 a 6)
    if opcion.isdigit():
        opcion = int(opcion)
        if opcion >= 1 and opcion <= 6:
            if opcion == 1:
                # Opción 1: Insertar un nodo
                arbol_propietarios.insertar_nodo_desde_usuario()

            elif opcion == 2:
                # Opción 2: Buscar un nodo por su métrica
                while True:
                    try:
                        metrica = int(input("Ingrese la métrica del nodo a buscar: "))
                        # Validar que la métrica sea un número
                        if not isinstance(metrica, int):
                            raise ValueError("La métrica debe ser un número válido.")
                        nodo_encontrado = arbol_propietarios.buscar_por_metrica(metrica)
                        break  # Salir del bucle si la métrica es válida y se realiza la búsqueda
                    except ValueError as e:
                        print(f"Error: {e}. Intente nuevamente.")

                if nodo_encontrado is None:
                    print("No existe nodo con esa metrica")
                else:
                    prelista_nodos.append(nodo_encontrado)
                    adicional = int(input("Mostrando información adicional. Digite: 1.Nivel, 2.Factor de balanceo, 3.Familia (Padre, abuelo y tío): "))
                    if adicional >= 1 and adicional <= 3:
                        if adicional == 1:
                            print("El nivel es: ", arbol_propietarios.calcular_nivel(nodo_encontrado.metrica_original))
                        elif adicional == 2:
                            print("El factor de balanceo es: ", arbol_propietarios.obtener_balance_factor(nodo_encontrado.metrica_original))
                        elif adicional == 3:
                            lista = arbol_propietarios.buscar_abuelo_padre_tio(nodo_encontrado.metrica_original)
                            for nodo in lista:
                                if nodo is not None:
                                    if nodo not in prelista_nodos:
                                        prelista_nodos.append(nodo)
                                    print(nodo.info)




            elif opcion == 3:
                # Opción 3: Eliminar un nodo por su métrica
                metrica = int(input("Ingrese la métrica del nodo a eliminar: "))
                arbol_propietarios.eliminar_por_metrica(metrica)




            elif opcion == 4:
                # Opción 4: Buscar un nodo por la serie de criterios
                menu_opcion4()
                for nodo in lista_nodos:
                    print(nodo.info)
                adicional = int(input("Mostrando información adicional. Digite: 1.Nivel, 2.Factor de balanceo, 3.Familia (Padre, abuelo y tío): "))
                if adicional >= 1 and adicional <= 3:
                    if adicional == 1:
                        for nodo in lista_nodos:
                            print("El nivel es: ", arbol_propietarios.calcular_nivel(nodo.metrica_original))
                    elif adicional == 2:
                        for nodo in lista_nodos:
                            print("El nivel es: ", arbol_propietarios.obtener_balance_factor(nodo.metrica_original))
                    elif adicional == 3:
                        for nodo in lista_nodos:
                            if nodo is not None:
                                lisas = arbol_propietarios.buscar_abuelo_padre_tio(nodo.metrica_original)
                            for lisa in lisas:
                                if lisa is not None:
                                    if nodo not in prelista_nodos:
                                        prelista_nodos.append(nodo)
                                    print(lisa.info)


                pass
            elif opcion == 5:
                # Opción 5: Mostrar árbol por niveles
                arbol_propietarios.recorrido_niveles_recursivo()

            elif opcion == 6:
                # Opción 6: Salir
                print("Saliendo del programa.")
                break
        else:
            print("La opción debe estar en el rango de 1 a 6.")
    else:
        print("Ingrese un número válido como opción (1/2/3/4/5/6).")




 
Menú de Opciones:
1. Insertar un nodo.
2. Buscar un nodo por su métrica.
3. Eliminar un nodo por su métrica
4. Buscar un nodo por la serie de criterios
5. Mostrar árbol por niveles.
6. Salir
Seleccione una opción (1/2/3/4/5/6): 2
Ingrese la métrica del nodo a buscar: 7647
Mostrando información adicional. Digite: 1.Nivel, 2.Factor de balanceo, 3.Familia (Padre, abuelo y tío): 3
{'titulo': 'Apartamento En Arriendo En Floridablanca Zapamanga Cod. ACLV-6242', 'departamento': 'Santander', 'ciudad': 'Floridablanca', 'tipo_inmueble': 'Apartamento', 'latitud': 7.067, 'longitud': -73.1, 'superficie_total': 98.0, 'superficie_cubierta': 98.0, 'cuartos': 5.0, 'banos': 2.0, 'tipo_operacion': 'Arriendo', 'precio': 800000.0}
{'titulo': 'Casa En Arriendo/venta En Cali Cascajal Corregimiento El Hormiguero Alto Maraon Cod. AJMI6360', 'departamento': 'Valle del Cauca', 'ciudad': 'Cali', 'tipo_inmueble': 'Casa', 'latitud': 3.261, 'longitud': -76.545, 'superficie_total': 2000.0, 'superficie_cubierta': 43

## **BONO**

In [ ]:
for nodo in prelista_nodos:
    print(nodo.info)

In [ ]:
for nodo in lista_nodos:
    print(nodo.info)

In [889]:
# Crear un mapa centrado en las coordenadas
mapa = folium.Map(location = [4.570868, -74.297333], zoom_start=5)

for i in range(len(lista_nodos)):
    popup_text = f"Precio: {lista_nodos[i].info['precio']}, Superficie Total: {lista_nodos[i].info['superficie_total']}, Cuartos: {lista_nodos[i].info['cuartos']}, Baños: {lista_nodos[i].info['banos']} "
    print(popup_text)
    folium.Marker([ lista_nodos[i].info['latitud'], lista_nodos[i].info['longitud'] ], tooltip= f"{lista_nodos[i].info['titulo']},\n{nodo.metrica_original} ", popup=popup_text).add_to(mapa)

for i in range(len(prelista_nodos)):
    popup_text = f"Precio: {prelista_nodos[i].info['precio']}, Superficie Total: {prelista_nodos[i].info['superficie_total']}, Cuartos: {prelista_nodos[i].info['cuartos']}, Baños: {prelista_nodos[i].info['banos']} "
    print(popup_text)
    folium.Marker([prelista_nodos[i].info['latitud'],prelista_nodos[i].info['longitud'] ], tooltip= f"{prelista_nodos[i].info['titulo']},\n{nodo.metrica_original} ", popup=popup_text).add_to(mapa)
# Mostrar el mapa

mapa


Precio: 4500000000.0, Superficie Total: 1900.0, Cuartos: 5.0, Baños: 7.0 
Precio: 3600000.0, Superficie Total: 350.0, Cuartos: 6.0, Baños: 7.0 
Precio: 650000.0, Superficie Total: 85.0, Cuartos: 3.0, Baños: 1.0 
Precio: 800000.0, Superficie Total: 98.0, Cuartos: 5.0, Baños: 2.0 
Precio: 3500000.0, Superficie Total: 2000.0, Cuartos: 4.0, Baños: 5.0 
Precio: 550000.0, Superficie Total: 56.0, Cuartos: 3.0, Baños: 1.0 
Precio: 4500000000.0, Superficie Total: 1900.0, Cuartos: 5.0, Baños: 7.0 
Precio: 3600000.0, Superficie Total: 350.0, Cuartos: 6.0, Baños: 7.0 
